confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time
import _init_paths

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib.pyplot import imsave
%matplotlib inline

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
SHRECmodels = '/home/jguerry/workspace/SHRECmodels/'
# prototxt_path = SHRECmodels+'c14_kf3_var5_train_val.prototxt'
# weights_path =  SHRECmodels+'c14_kf3_var5.caffemodel'

# prototxt_path = SHRECmodels+'14bi28_kf5_var5 .prototxt'
# weights_path =  SHRECmodels+'14bi28_kf5_var5.caffemodel'

prototxt_path = SHRECmodels+'THEproto.prototxt'
weights_path =  SHRECmodels+'THEweights.caffemodel'





caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [4]:
# net = caffe.Net(prototxt_path, caffe.TEST)
net = caffe.Net(prototxt_path, weights_path, caffe.TEST)

In [5]:
results={}
results['id_gesture']=[]
results['id_finger']=[]
results['id_subject']=[]
results['id_essai']=[]
results['label_gt']=[]
results['label_found']=[]
results['label_gt_28']=[]
results['label_found_28']=[]
results['time_millisecond']=[]
                     

In [6]:
print("###############################################################")

# net.layers[0].images_processed = 100

for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg='Finished !',removeAtTheEnd=True):
# for idx in log_progress(range(5), title='iteration', end_msg='Finished !',removeAtTheEnd=True):

    #print("Idx :",idx)
    t = time.time()
    net.forward()
    results['id_gesture'].append(int(net.layers[0].list_images[idx,0]))
    results['id_finger'].append(int(net.layers[0].list_images[idx,1]))
    results['id_subject'].append(int(net.layers[0].list_images[idx,2]))
    results['id_essai'].append(int(net.layers[0].list_images[idx,3]))
    results['label_gt'].append(int(net.blobs['label'].data[0]))
#     results['label_found'].append(int(net.blobs['fc_14'].data[0].argmax(0)))
    results['label_gt_28'].append(int(net.blobs['label_28'].data[0]))
    results['label_found_28'].append(int(net.blobs['fc_28'].data[0].argmax(0)))
    results['time_millisecond'].append(round(100000*(time.time() - t))/100)




###############################################################


In [7]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [8]:
# hist = fast_hist(np.asarray(results['label_gt']),np.asarray(results['label_found']),14)


# acc = np.diag(hist).sum() / hist.sum()
# # per-class accuracy
# pc_acc = np.diag(hist) / hist.sum(1)
# mean_accuracy= np.nanmean(pc_acc)

# print("Accuracy:",acc)
# print("pc_acc:",pc_acc)
# print("mean_accuracy:",mean_accuracy)

In [9]:
hist = fast_hist(np.asarray(results['label_gt_28']),np.asarray(results['label_found_28']),28)


acc = np.diag(hist).sum() / hist.sum()
# per-class accuracy
pc_acc = np.diag(hist) / hist.sum(1)
mean_accuracy= np.nanmean(pc_acc)

print("Accuracy:",acc)
print("pc_acc:",pc_acc)
print("mean_accuracy:",mean_accuracy)

Accuracy: 0.719047619048
pc_acc: [ 0.67857143  0.83333333  0.3030303   0.5         0.59259259  0.75
  0.62962963  0.83333333  0.31034483  0.76923077  0.5         0.73076923
  0.66666667  0.86206897  0.69230769  0.75        0.8125      0.80555556
  0.61290323  0.86666667  0.60606061  0.88888889  0.75        0.96551724
  0.89285714  0.9         0.78947368  0.82857143]
mean_accuracy: 0.718602614734


## convert 28 to 14

In [10]:
cl28tocl14 = np.zeros(840,dtype=np.uint8)
for idx,cl_28 in enumerate(results['label_found_28']):
    cl_14 = int((float(cl_28-(cl_28%2)))/2)
    #print(cl_28,'becomes',cl_14)
    cl28tocl14[idx]=cl_14

In [11]:
hist = fast_hist(np.asarray(results['label_gt']),cl28tocl14,14)
acc = np.diag(hist).sum() / hist.sum()
# per-class accuracy
pc_acc = np.diag(hist) / hist.sum(1)
mean_accuracy= np.nanmean(pc_acc)

print("Accuracy:",acc)
print("pc_acc:",pc_acc)
print("mean_accuracy:",mean_accuracy)

Accuracy: 0.828571428571
pc_acc: [ 0.86206897  0.49180328  0.76363636  0.76470588  0.78181818  0.94827586
  0.9516129   0.83333333  0.82352941  0.8852459   0.7826087   0.92982456
  0.94827586  0.83561644]
mean_accuracy: 0.82873968868


In [12]:
print("Mean time per sequence :",int(round(np.mean(results['time_millisecond']))),'ms')

Mean time per sequence : 236 ms


## Sauvegarde des fichiers resultats

#### id_gesture     id_finger    id_subject    id_essai    label_found    time_millisecond

In [16]:
resultsfile14 = open('/home/jguerry/workspace/SHRECmodels/results14.txt','w+')
resultsfile28 = open('/home/jguerry/workspace/SHRECmodels/results28.txt','w+')
readmefiles = open('/home/jguerry/workspace/SHRECmodels/readme.txt','w+')

In [17]:
for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg='Finished !',removeAtTheEnd=True):
    
    id_gesture = results['id_gesture'][idx]
    id_finger = results['id_finger'][idx]
    id_subject = results['id_subject'][idx]
    id_essai = results['id_essai'][idx]
    label_found_28 = results['label_found_28'][idx]+1
    label_found_14 = cl28tocl14[idx]+1
    time_millisecond = results['time_millisecond'][idx]
    
    
    print(id_gesture,id_finger,id_subject,id_essai,label_found_14,time_millisecond, file=resultsfile14)
    print(id_gesture,id_finger,id_subject,id_essai,label_found_28,time_millisecond, file=resultsfile28)
print("\
       +-----------------------------------------------------------------------------+\n\
       |  GeForce GTX 1070   Driver Version: 375.26   8112MiB                        |\n\
       +-----------------------------------------------------------------------------+\n\
       |  Intel(R) Xeon(R) CPU E5-1620 v4 @ 3.50GHz                                  |\n\
       |  32GB RAM                                                                   |\n\
       +-----------------------------------------------------------------------------+\n\
", file=readmefiles)

In [18]:
resultsfile14.close()
resultsfile28.close()
readmefiles.close()